#Grupo 25 - Entregable Aprendizaje Supervisado

#Parte 2 de 3

Integrantes:

Rufeil Fiori, Elena

Sosa, Manuel

Taié Mirón, Macarena

Yurovich, Luis Federico

# Preprocesamiento

Resumen:

- Ingenieria de características: Separación Cabin, agregado de cabin_qty y total_ammenities
- Filtrado de valores extremos: no efectuamos
- Codificación de valores categóricos
- Escalado 
- Imputación de valores nulos



Importamos las librerías necesarias para el prepocesamiento

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
url_test = 'https://raw.githubusercontent.com/DiploDatos/AprendizajeSupervisado/master/Pr%C3%A1ctico/test.csv'
url_train = 'https://raw.githubusercontent.com/DiploDatos/AprendizajeSupervisado/master/Pr%C3%A1ctico/train.csv'
df_train = pd.read_csv(url_train, sep = ',')
df_test = pd.read_csv(url_test, sep = ',')

Generamos una copia del data frame del conjunto de entrenamiento, al que le aplicaremos el preprocesamiento. Posteriormente, realizamos los mismos pasos al conjunto de test, cambiando la asignación del "df" inicial.

In [ ]:
df = df_train.copy()
#df = df_test.copy()
df.sample(10)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
2656,5488,Europa,True,B/199/P,55 Cancri e,NaN,False,0.0,0.0,0.0,0.0,0.0,Graviph Wincer,1
2857,1872,Earth,NaN,G/315/S,TRAPPIST-1e,42.0,False,37.0,1670.0,0.0,0.0,0.0,Joyn Hartley,1
5785,3126,Earth,False,F/694/P,TRAPPIST-1e,42.0,False,0.0,3.0,0.0,898.0,1.0,Marix Birds,0
3785,3088,Earth,False,F/691/P,TRAPPIST-1e,34.0,False,578.0,0.0,0.0,0.0,323.0,Rena Yorkland,0
5802,1557,Earth,True,G/266/P,PSO J318.5-22,0.0,False,0.0,0.0,0.0,0.0,0.0,Deanny Wonglasquez,1
7252,7499,Europa,False,D/249/P,TRAPPIST-1e,14.0,False,0.0,4545.0,271.0,199.0,1377.0,Cella Mallisly,1
5264,4110,Mars,True,F/903/P,TRAPPIST-1e,8.0,False,0.0,0.0,0.0,0.0,0.0,Jayrin Vloaf,1
5034,7140,Earth,False,F/1457/S,PSO J318.5-22,20.0,False,11.0,725.0,0.0,0.0,0.0,Jandy Boyersons,1
5041,8014,Mars,False,F/1653/S,TRAPPIST-1e,54.0,False,1763.0,0.0,11.0,0.0,0.0,Ceros Welte,0
2195,250,Earth,False,F/61/P,55 Cancri e,50.0,False,44.0,2390.0,0.0,53.0,0.0,Byront Mcdanield,1


Convertimos en index a la variable PassengerId para no involucrarla como input en los modelos de machine learning, pero al mismo tiempo conservar el vínculo con sus atributos.

In [ ]:
df = df.set_index("PassengerId")
df.sample(10)

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
PassengerId,,,,,,,,,,,,,
7633,Earth,True,G/1319/P,PSO J318.5-22,26.0,False,0.0,0.0,0.0,0.0,0.0,Gordo Contrevins,0
2470,Earth,False,G/421/S,TRAPPIST-1e,23.0,False,490.0,313.0,0.0,6.0,0.0,Allena Sullones,0
3753,Europa,False,B/125/P,TRAPPIST-1e,42.0,False,0.0,456.0,1.0,1193.0,3039.0,Hekabih Metful,0
798,Earth,NaN,G/123/P,TRAPPIST-1e,27.0,False,663.0,1111.0,0.0,0.0,2.0,Ebonyd Braymon,0
3063,Mars,True,D/106/P,TRAPPIST-1e,35.0,False,0.0,0.0,0.0,0.0,0.0,Swink Jaff,0
1034,Mars,False,F/226/P,TRAPPIST-1e,5.0,False,NaN,0.0,0.0,0.0,0.0,Jel Cuche,1
6854,NaN,False,E/482/S,TRAPPIST-1e,21.0,False,1046.0,1.0,0.0,1.0,0.0,Poloos Burle,0
7619,Earth,True,G/1315/P,55 Cancri e,20.0,False,0.0,0.0,0.0,0.0,0.0,Nica Whitneyes,0
6626,Earth,False,NaN,PSO J318.5-22,21.0,False,1.0,22.0,0.0,649.0,2.0,Lison Franton,0


## Ingeniería de características

A partir de la variable "Cabin" agregamos tres atributos: "Deck", "Num" y "Side" que corresponden a la cubierta, número de cabina y lado de la nave (babor y estribor). Además, generamos una columna que corresponde a la cantidad de personas que se encuentran en dicha cabina: "Cabin_qty".

In [ ]:
df = df.merge(df.Cabin.str.split('/', expand = True).rename(columns = dict(zip([0,1,2],["Deck", "Num", "Side"]))), 
         left_index=True, right_index=True)
df["Num"] = df["Num"].astype(float)

In [ ]:
cabin_groups = df.Cabin.value_counts()
df["Cabin_qty"] = df[df.Cabin.notnull()].Cabin.apply(lambda c: cabin_groups.loc[c])

Por otro lado, generamos una nueva variable, "Total Ammenities", a partír de la suma de las variables de los servicios de lujo: "RoomService", "FoodCourt", "ShoppingMall", "Spa" y "VRDeck".

In [ ]:
ammenities_columns = ["RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"]
df["Total_ammenities"] = df[ammenities_columns].sum(axis = 1)

## Filtrado de valores extremos

Decidimos no filtrar outliers ya que consideramos que estos datos caracterizan a la población, brindando información para nuestros modelos.

## Codificación de valores categóricos

Para la variable "Deck" aplicamos una transformación ordinal que convierte de {A, B, C, D, E, F, G, T} a {0, 1, 2, 3, 4, 5, 6, 7}, respectivamente.

Las variables binarias "VIP" con valores {False, True}, "Side" con {S, P} y "CryoSleep" con {False, True}, son transformadas a {0, 1}, respectivamente.

Las variables "HomePlanet" y "Destination" fueron transformadas utilizando el método One Hot Encoding. En estos casos, hemos optado por dejar los valores nulos de estas variables como una nueva categoría, ya que creemos que la imputación en estas variables podría generar ruido.

In [ ]:
df_processed = df.copy()
#df_processed.index.name = "PassengerId"
df_processed

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Deck,Num,Side,Cabin_qty,Total_ammenities
PassengerId,,,,,,,,,,,,,,,,,,
3430,Europa,False,C/123/P,TRAPPIST-1e,57.0,False,0.0,3859.0,13.0,195.0,3874.0,Azhah Tronsive,0,C,123.0,P,4.0,7941.0
669,Mars,True,F/126/S,NaN,18.0,False,0.0,0.0,0.0,0.0,0.0,Roswal Sha,1,F,126.0,S,1.0,0.0
8430,Europa,True,B/343/S,TRAPPIST-1e,33.0,NaN,0.0,0.0,0.0,0.0,0.0,Shaiam Tinganking,1,B,343.0,S,1.0,0.0
6463,Earth,False,F/1420/P,TRAPPIST-1e,43.0,False,142.0,1208.0,0.0,0.0,204.0,Landa Grifford,0,F,1420.0,P,1.0,1554.0
85,Earth,False,G/9/P,TRAPPIST-1e,19.0,False,0.0,0.0,670.0,1.0,34.0,Gracy Dunnisey,0,G,9.0,P,3.0,705.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6528,Mars,False,F/1319/S,TRAPPIST-1e,12.0,NaN,0.0,0.0,0.0,0.0,0.0,Risps Hacle,1,F,1319.0,S,3.0,0.0
2693,Earth,False,G/459/S,TRAPPIST-1e,44.0,False,0.0,0.0,744.0,62.0,3.0,Gabrie Reenez,0,G,459.0,S,1.0,809.0
8076,Mars,True,F/1772/P,TRAPPIST-1e,55.0,False,0.0,0.0,0.0,0.0,0.0,Panzel Ruity,1,F,1772.0,P,1.0,0.0


In [ ]:
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()

In [ ]:
deck_ordinal = enc.fit_transform(df["Deck"].values.reshape(-1,1))
df_processed["Deck"] = pd.Series(deck_ordinal.flatten())

In [ ]:
side_encoded = enc.fit_transform(df_processed["Side"].values.reshape(-1,1))
df_processed["Side"] = pd.Series(side_encoded.flatten())

In [ ]:
for c in ["VIP", "CryoSleep"]:
  df_processed[c] = df_processed[c].astype(float)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

one_hot_enc = OneHotEncoder()

In [ ]:
homeplanet_enc = one_hot_enc.fit_transform(df["HomePlanet"].values.reshape(-1,1))
df_processed = df_processed.merge(pd.DataFrame(homeplanet_enc.toarray(), 
                                columns = [f"HomePlanet_{cat}" for cat in one_hot_enc.categories_[0]]), left_index = True, right_index = True)

In [ ]:
destination_enc = one_hot_enc.fit_transform(df["Destination"].values.reshape(-1,1))
df_processed = df_processed.merge(pd.DataFrame(destination_enc.toarray(), 
                                columns = [f"Destination_{cat}" for cat in one_hot_enc.categories_[0]]), left_index = True, right_index = True)

In [ ]:
df_processed['Side']

3430    1.0
669     1.0
6463    0.0
85      0.0
7586    1.0
       ... 
2204    1.0
6528    0.0
2693    0.0
3829    1.0
7624    0.0
Name: Side, Length: 7047, dtype: float64

Continuando con la ingeniería de características, eliminamos del data frame las variables "HomePlanet", "Destination" y "Cabin", ya que al tener su codificación correspondiente, no es necesario conservarlas. Asimismo, eliminamos la variable "Name" ya que consideramos que su información no es relevante para la predicción.

In [ ]:
df_processed = df_processed.drop(columns = ["HomePlanet", "Destination", "Cabin", "Name"])

## Escalado de los datos

Elegimos escalar los datos usando el método MaxAbsScaler, debido a que este método no centra los datos y por lo tanto es una mejor opción para datos esparsos que el método más usado MinMaxScaler.

In [ ]:
from sklearn.preprocessing import MaxAbsScaler
scaler = MaxAbsScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df_processed), columns = df_processed.columns)

In [ ]:
df2 = df_scaled

##  Imputación de valores nulos


Las siguientes variables son imputadas por su valor más frecuente:
- RoomService
- FoodCourt
- ShoppingMall
- Spa
- VRDeck
- CryoSleep
- VIP
- Cabin_qty

Mientras que para las siguientes variables, utilizamos el método KNN Imputer:
- Age
- Deck
- Num
- Side

In [ ]:
imputar_mas_frecuente = ['CryoSleep', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', "Cabin_qty"]

In [ ]:
from sklearn.impute import SimpleImputer

simple = SimpleImputer(strategy='most_frequent')

mode = simple.fit_transform(df2.loc[:, imputar_mas_frecuente])

df2.loc[:, imputar_mas_frecuente] = pd.DataFrame(mode, columns = imputar_mas_frecuente)


In [ ]:
col_knn_imputer = ["Age", "Deck", "Num", "Side"]

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.impute import KNNImputer
knn_imputer = KNNImputer(n_neighbors=5, weights="uniform")

In [ ]:
df2 = pd.DataFrame(knn_imputer.fit_transform(df2),
             columns = df2.columns)

In [ ]:
df2

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Deck,...,Cabin_qty,Total_ammenities,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,HomePlanet_nan,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Destination_nan
0,0.0,0.721519,0.0,0.000000,0.129440,0.001061,0.008702,0.160527,0.0,0.571429,...,0.500,0.220663,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1.0,0.227848,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.714286,...,0.125,0.000000,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.544304,0.0,0.009911,0.040519,0.000000,0.000000,0.008453,0.0,0.714286,...,0.125,0.043182,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.240506,0.0,0.000000,0.000000,0.054680,0.000045,0.001409,0.0,0.714286,...,0.375,0.019590,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.291139,0.0,0.006282,0.000134,0.000000,0.000312,0.024365,0.0,0.857143,...,0.125,0.019146,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7042,1.0,0.303797,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.714286,...,0.125,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7043,0.0,0.151899,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.714286,...,0.375,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7044,0.0,0.556962,0.0,0.000000,0.000000,0.060720,0.002767,0.000124,0.0,0.714286,...,0.125,0.022480,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7045,0.0,0.189873,0.0,0.000000,0.028176,0.010038,0.000000,0.000829,0.0,0.714286,...,0.125,0.027315,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


# Guardamos los data frames preprocesados

In [ ]:
#df.to_csv("/content/drive/MyDrive/DIPLODATOS 2022 grupo 25/Entregables/4 - Aprendizaje Supervisado/df test procesado.csv", index = True)

In [ ]:
#df.to_csv("/content/drive/MyDrive/DIPLODATOS 2022 grupo 25/Entregables/4 - Aprendizaje Supervisado/df train procesado.csv", index = True)